# Evaluate SR output - hallucations and faithfullness

In [1]:
import sys

sys.path = ['/Users/jonathan/Projects/semantic-retrieval/python/src'] + sys.path

## Reproduce reports

In [2]:
# in semantic-retrieval/
# Reproduce net income report
!python python/src/semantic_retrieval/examples/financial_report/generate_report.py --retrieval-query="annual net income 2022" --data-extraction-prompt="annual net income in 2022" --overfetch-factor=20

python: can't open file '/Users/jonathan/Projects/semantic-retrieval/python/src/semantic_retrieval/examples/financial_report/python/src/semantic_retrieval/examples/financial_report/generate_report.py': [Errno 2] No such file or directory


In [3]:
# Reproduce covid report
# in semantic-retrieval/
!python python/src/semantic_retrieval/examples/financial_report/generate_report.py --retrieval-query="covid 19 impact" --data-extraction-prompt="covid 19 impact" --overfetch-factor=20 --log-level=DEBUG --client-name=client_a

python: can't open file '/Users/jonathan/Projects/semantic-retrieval/python/src/semantic_retrieval/examples/financial_report/python/src/semantic_retrieval/examples/financial_report/generate_report.py': [Errno 2] No such file or directory


## Example eval input: 
* Client portfolio database (local CSV)
* Intermediate SR results: raw retrieved document chunks
* SR final output saved in local files

In [16]:
import sys
import pandas as pd

from semantic_retrieval.examples.financial_report.evaluate_report import file_contents
from semantic_retrieval.retrieval.csv_retriever import CSVRetriever
from semantic_retrieval.common import types
from semantic_retrieval.evaluation.lib import (
    LocalFileSystemGenLLMEvalDataset,
    SampleEvalDataset,
    local_filesystem_dataset_to_df,
)

import glob
import os




ROOT_DATA_DIR = "../../../../../examples/example_data/financial_report/"

ARTIFACTS = os.path.join(ROOT_DATA_DIR, "artifacts")

### Define evaluation test cases
* The first two test the final output against the raw retrieved data
  for faithfullness (LLM hallucination), and the second two test e2e
  for faithfullness against the input structured data (portfolio).
* In a production system, the equivalent of this
    would be written to an RDB system automatically
    during retrieval system operation.

## Configuration & Customization

In [23]:
TEST_CASES = [
    (name, os.path.join(ARTIFACTS, ip), os.path.join(ARTIFACTS, op))
    for name, ip, op in [
        ("net_income_vs_retrieved", "raw_retrieved_chunks_10k_net_income.json", "portfolio_10k_net_income_report.txt"),
        ("covid_vs_retrieved", "raw_retrieved_chunks_10k_covid.json", "portfolio_10k_covid_report.txt"),
    ]
] + [
    (name, os.path.join(ROOT_DATA_DIR, ip), os.path.join(ARTIFACTS, op))
    for name, ip, op in [
        ("net_income_e2e", "portfolios/client_a_portfolio.csv", "portfolio_10k_net_income_report.txt"),
        ("covid_e2e", "portfolios/client_a_portfolio.csv", "portfolio_10k_covid_report.txt"),                    
    ]
]

In [6]:
# Customize the raw data -> structured data step
def portfolio_test_path_muncher(in_content, out_content) -> PortfolioLLMEvalStructuredSample:
    pass

def portfolio_faithfullness(portfolio_structured_dataset: PortfolioLLMEvalStructuredSample) -> bool:
    pass



## Evaluation and inspection using LM-SR library

In [7]:
portfolio = await CSVRetriever("../../../../../examples/example_data/financial_report/portfolios/client_a_portfolio.csv")\
    .retrieve_data(None)

portfolio

{'AAPL': 20.0,
 'AMZN': 30.0,
 'NVDA': 100.0,
 'UNH': 30.0,
 'JNJ': 100.0,
 'MRK': 40.0,
 'PEP': 200.0}

In [25]:
llm_eval_dataset = LocalFileSystemGenLLMEvalDataset.from_list(TEST_CASES)

### Optional: peek into the contents manually for better visibility.

In [26]:
df = local_filesystem_dataset_to_df(llm_eval_dataset)

pd.set_option("display.max_colwidth", 200)
display(df)

,name,data_input,data_output
0,net_income_vs_retrieved,"[\n {\n ""company"": ""AAPL"",\n ""details"": ""net (\""OI&E\"") for 2022, 2021 and 2020 was as follows (dollars in millions): | | | | | | | | | | | | | | | | | | | | | | | | | | | | | ---|---...","Report:\n\n1. AAPL's annual net income in 2022: $2,825.\n2. PEP's annual net income in 2022: $86,392.\n3. AMZN's annual net income in 2022: $1.3 billion.\n4. JNJ's annual net income in 2022: -$1,7..."
1,covid_vs_retrieved,"[\n {\n ""company"": ""NVDA"",\n ""details"": ""The COVID-19 pandemic has affected and could continue to have a material adverse impact on our financial condition and results of operations. ...","1. Apple (AAPL): The COVID-19 pandemic has had a significant impact on Apple's business, with restrictions on travel and business operations, temporary closures of businesses, and supply shortages..."
2,net_income_e2e,"Company,Shares,,,,,\nAAPL,20,,,,,\nMSFT,,,,,,\nAMZN,30,,,,,\nNVDA,100,,,,,\nTSLA,,,,,,\nGOOG,,,,,,\nBRK.B,,,,,,\nMETA,,,,,,\nUNH,30,,,,,\nXOM,,,,,,\nLLY,,,,,,\nJPM,,,,,,\nJNJ,100,,,,,\nV,,,,,,\nPG...","Report:\n\n1. AAPL's annual net income in 2022: $2,825.\n2. PEP's annual net income in 2022: $86,392.\n3. AMZN's annual net income in 2022: $1.3 billion.\n4. JNJ's annual net income in 2022: -$1,7..."
3,covid_e2e,"Company,Shares,,,,,\nAAPL,20,,,,,\nMSFT,,,,,,\nAMZN,30,,,,,\nNVDA,100,,,,,\nTSLA,,,,,,\nGOOG,,,,,,\nBRK.B,,,,,,\nMETA,,,,,,\nUNH,30,,,,,\nXOM,,,,,,\nLLY,,,,,,\nJPM,,,,,,\nJNJ,100,,,,,\nV,,,,,,\nPG...","1. Apple (AAPL): The COVID-19 pandemic has had a significant impact on Apple's business, with restrictions on travel and business operations, temporary closures of businesses, and supply shortages..."


In [10]:
# Simple regexp parsing from LLM's structured output

from semantic_retrieval.examples.financial_report.evaluate_report import gen_output_to_df
    
df_report_parsed = gen_output_to_df(report)
df_report_parsed

NameError: name 'report' is not defined

In [ ]:
import json

gen_model_input_data = json.loads(file_contents("../../../../../examples/example_data/financial_report/artifacts/raw_retrieved_chunks_10k.json"))

print(gen_model_input_data[0])

## Check ticker agreement

In [ ]:
input_companies = set([input_record['company'] for input_record in gen_model_input_data])
output_companies = set(df_report_parsed.ticker)

assert input_companies == output_companies

print("Pass")

## Check number faithfulness

In [ ]:
hallucinations = 0

for company, value_raw in df_report_parsed.set_index("ticker").value_raw.to_dict().items():
    # print(company, value_raw)
    input_data_for_company = [
        input_record["details"] for input_record in gen_model_input_data
        if company == input_record["company"]
    ][0]

    # Remove the period that the LLM might have put on to end the sentence.
    if value_raw.rstrip(".") not in input_data_for_company:
        hallucinations += 1

    # print(value_raw)
    # print(input_data_for_company)

hr = round(hallucinations / len(df_report_parsed), 2)
print(f"Hallucination rate: {100.0 * hr}%")

## E2e: Check final output's faithfulness to portfolio (structured table)

In [ ]:
assert set(df_report_parsed.ticker) == portfolio.keys()

print("Pass")